# Multi-Model Synthetic Data Generation: Breast Cancer Dataset

## Comprehensive Demo and Hyperparameter Tuning of 5 Models

This notebook demonstrates and hypertunesall 5 available models:
1. **CTGAN** - Conditional Tabular GAN
2. **TVAE** - Tabular Variational AutoEncoder  
3. **CopulaGAN** - Copula-based GAN
4. **GANerAid** - Enhanced GAN with clinical focus
5. **TableGAN** - Table-specific GAN implementation

### Methodology:
1. **Phase 1**: Demo each model with default parameters
2. **Phase 2**: Hypertune each model individually
3. **Phase 3**: Identify best hyperparameters per model
4. **Phase 4**: Re-tune best models with optimal parameters
5. **Phase 5**: Compare all models and identify overall best
6. **Phase 6**: Comprehensive analysis and visualizations

### Dataset: Breast Cancer Wisconsin (Diagnostic)
- **Features**: 5 continuous variables + 1 binary target
- **Target**: Diagnosis (0=benign, 1=malignant)
- **Samples**: 569 rows
- **Use Case**: Medical diagnosis classification

## Setup and Configuration

In [ ]:
# Enhanced imports for multi-model analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import os
from datetime import datetime
import json
import time
from typing import Dict, List, Tuple, Any

# Model imports
try:
    from src.models.model_factory import ModelFactory
    from src.evaluation.unified_evaluator import UnifiedEvaluator
    from src.optimization.optuna_optimizer import OptunaOptimizer
    FRAMEWORK_AVAILABLE = True
    print("✅ Multi-model framework imported successfully")
except ImportError as e:
    print(f"⚠️ Framework import failed: {e}")
    print("📋 Will use individual model imports")
    FRAMEWORK_AVAILABLE = False

# Individual model imports as fallback
MODEL_STATUS = {}

# CTGAN
try:
    from src.models.implementations.ctgan_model import CTGANModel
    MODEL_STATUS['CTGAN'] = True
    print("✅ CTGAN available")
except ImportError:
    MODEL_STATUS['CTGAN'] = False
    print("⚠️ CTGAN not available")

# TVAE
try:
    from src.models.implementations.tvae_model import TVAEModel
    MODEL_STATUS['TVAE'] = True
    print("✅ TVAE available")
except ImportError:
    MODEL_STATUS['TVAE'] = False
    print("⚠️ TVAE not available")

# CopulaGAN
try:
    from src.models.implementations.copulagan_model import CopulaGANModel
    MODEL_STATUS['CopulaGAN'] = True
    print("✅ CopulaGAN available")
except ImportError:
    MODEL_STATUS['CopulaGAN'] = False
    print("⚠️ CopulaGAN not available")

# GANerAid
try:
    from src.models.implementations.ganeraid_model import GANerAidModel
    MODEL_STATUS['GANerAid'] = True
    print("✅ GANerAid available")
except ImportError:
    MODEL_STATUS['GANerAid'] = False
    print("⚠️ GANerAid not available")

# TableGAN
try:
    from src.models.implementations.tablegan_model import TableGANModel
    MODEL_STATUS['TableGAN'] = True
    print("✅ TableGAN available")
except ImportError:
    MODEL_STATUS['TableGAN'] = False
    print("⚠️ TableGAN not available")

# Optimization framework
try:
    import optuna
    from optuna.samplers import TPESampler
    OPTUNA_AVAILABLE = True
    print("✅ Optuna optimization available")
except ImportError:
    OPTUNA_AVAILABLE = False
    print("⚠️ Optuna not available - will use basic grid search")

# Evaluation libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy import stats

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
np.random.seed(42)

# Create results directory
RESULTS_DIR = Path('results/multi_model_analysis')
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Export configuration
EXPORT_FIGURES = True
EXPORT_TABLES = True
FIGURE_FORMAT = 'png'
FIGURE_DPI = 300

print(f"\n📊 MULTI-MODEL FRAMEWORK STATUS:")
available_models = [model for model, status in MODEL_STATUS.items() if status]
unavailable_models = [model for model, status in MODEL_STATUS.items() if not status]

print(f"✅ Available models ({len(available_models)}): {', '.join(available_models)}")
if unavailable_models:
    print(f"⚠️ Unavailable models ({len(unavailable_models)}): {', '.join(unavailable_models)}")

print(f"\n📁 Results directory: {RESULTS_DIR.absolute()}")
print(f"📊 Export settings - Figures: {EXPORT_FIGURES}, Tables: {EXPORT_TABLES}")
print(f"🔧 Optimization framework: {'Optuna' if OPTUNA_AVAILABLE else 'Basic Grid Search'}")

## Data Loading and Preprocessing

In [ ]:
# Load and preprocess breast cancer data
DATA_FILE = "data/Breast_cancer_data.csv"
TARGET_COLUMN = "diagnosis"
DATASET_NAME = "Breast Cancer Wisconsin (Diagnostic)"

print(f"📊 LOADING {DATASET_NAME}")
print("="*50)

try:
    # Load data
    data = pd.read_csv(DATA_FILE)
    print(f"✅ Data loaded successfully: {data.shape}")
    
    # Basic data info
    print(f"\n📋 Dataset Overview:")
    print(f"   • Shape: {data.shape[0]} rows × {data.shape[1]} columns")
    print(f"   • Missing values: {data.isnull().sum().sum()}")
    print(f"   • Duplicate rows: {data.duplicated().sum()}")
    print(f"   • Memory usage: {data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Target analysis
    if TARGET_COLUMN in data.columns:
        target_counts = data[TARGET_COLUMN].value_counts().sort_index()
        print(f"\n🎯 Target Variable ({TARGET_COLUMN}):")
        for value, count in target_counts.items():
            percentage = (count / len(data)) * 100
            label = 'Benign' if value == 0 else 'Malignant' if value == 1 else f'Class {value}'
            print(f"   • {label} ({value}): {count} samples ({percentage:.1f}%)")
        
        balance_ratio = target_counts.min() / target_counts.max()
        balance_status = 'Balanced' if balance_ratio > 0.8 else 'Moderately Imbalanced' if balance_ratio > 0.5 else 'Highly Imbalanced'
        print(f"   • Balance ratio: {balance_ratio:.3f} ({balance_status})")
    
    # Data preprocessing
    print(f"\n🔧 Preprocessing data...")
    processed_data = data.copy()
    
    # Handle missing values (if any)
    missing_counts = processed_data.isnull().sum()
    if missing_counts.sum() > 0:
        print(f"   • Handling {missing_counts.sum()} missing values")
        for col in missing_counts[missing_counts > 0].index:
            if processed_data[col].dtype in ['int64', 'float64']:
                processed_data[col].fillna(processed_data[col].median(), inplace=True)
            else:
                processed_data[col].fillna(processed_data[col].mode()[0], inplace=True)
    else:
        print(f"   • No missing values to handle")
    
    # Remove duplicates (if any)
    duplicates = processed_data.duplicated().sum()
    if duplicates > 0:
        processed_data = processed_data.drop_duplicates()
        print(f"   • Removed {duplicates} duplicate rows")
    else:
        print(f"   • No duplicates to remove")
    
    # Data type optimization
    print(f"   • Optimizing data types")
    for col in processed_data.select_dtypes(include=['int64']).columns:
        processed_data[col] = pd.to_numeric(processed_data[col], downcast='integer')
    for col in processed_data.select_dtypes(include=['float64']).columns:
        processed_data[col] = pd.to_numeric(processed_data[col], downcast='float')
    
    print(f"\n✅ Preprocessing completed: {processed_data.shape}")
    print(f"📋 Final dataset ready for multi-model analysis")
    
    # Display sample
    print(f"\n📋 Sample data:")
    display(processed_data.head())
    
    # Export preprocessed data
    if EXPORT_TABLES:
        processed_data.to_csv(RESULTS_DIR / 'preprocessed_breast_cancer_data.csv', index=False)
        print(f"💾 Preprocessed data exported: {RESULTS_DIR / 'preprocessed_breast_cancer_data.csv'}")
    
except FileNotFoundError:
    print(f"❌ Error: Could not find file {DATA_FILE}")
    raise
except Exception as e:
    print(f"❌ Error processing data: {e}")
    raise

## Phase 1: Demo All Models with Default Parameters

In [ ]:
# Phase 1: Demo all available models with default parameters
print("🚀 PHASE 1: DEMO ALL MODELS WITH DEFAULT PARAMETERS")
print("="*60)

# Initialize results storage
phase1_results = {}
phase1_synthetic_data = {}
phase1_training_times = {}
phase1_generation_times = {}

# Demo configuration
DEMO_EPOCHS = 1000  # Reduced for demo purposes
DEMO_SAMPLES = len(processed_data)

print(f"📊 Demo Configuration:")
print(f"   • Training epochs: {DEMO_EPOCHS:,}")
print(f"   • Samples to generate: {DEMO_SAMPLES:,}")
print(f"   • Models to demo: {len(available_models)}")
print(f"\n🎯 Starting model demonstrations...\n")

for model_name in available_models:
    print(f"🔧 DEMOING {model_name.upper()}")
    print("-" * 30)
    
    try:
        demo_start = time.time()
        
        # Initialize model with default parameters
        if model_name == 'CTGAN':
            model = CTGANModel()
            # CTGAN default parameters
            train_params = {
                'epochs': DEMO_EPOCHS,
                'batch_size': 500,
                'discriminator_lr': 2e-4,
                'generator_lr': 2e-4,
                'discriminator_decay': 1e-6,
                'generator_decay': 1e-6
            }
            
        elif model_name == 'TVAE':
            model = TVAEModel()
            # TVAE default parameters
            train_params = {
                'epochs': DEMO_EPOCHS,
                'batch_size': 500,
                'compress_dims': (128, 128),
                'decompress_dims': (128, 128),
                'l2scale': 1e-5,
                'learning_rate': 1e-3
            }
            
        elif model_name == 'CopulaGAN':
            model = CopulaGANModel()
            # CopulaGAN default parameters
            train_params = {
                'epochs': DEMO_EPOCHS,
                'batch_size': 500,
                'discriminator_lr': 2e-4,
                'generator_lr': 2e-4,
                'discriminator_decay': 1e-6,
                'generator_decay': 1e-6
            }
            
        elif model_name == 'GANerAid':
            model = GANerAidModel()
            # GANerAid default parameters
            train_params = {
                'epochs': DEMO_EPOCHS,
                'lr_d': 0.0005,
                'lr_g': 0.0005,
                'hidden_feature_space': 200,
                'batch_size': 100,
                'nr_of_rows': 25,
                'binary_noise': 0.2
            }
            
        elif model_name == 'TableGAN':
            model = TableGANModel()
            # TableGAN default parameters
            train_params = {
                'epochs': DEMO_EPOCHS,
                'batch_size': 32,
                'lr': 0.0002,
                'beta1': 0.5,
                'beta2': 0.999
            }
        
        print(f"   📊 Default parameters:")
        for param, value in train_params.items():
            print(f"      • {param}: {value}")
        
        # Train model
        print(f"   🚀 Training {model_name}...")
        training_start = time.time()
        
        model.fit(processed_data, **train_params)
        
        training_end = time.time()
        training_time = training_end - training_start
        phase1_training_times[model_name] = training_time
        
        print(f"   ✅ Training completed in {training_time:.2f} seconds")
        
        # Generate synthetic data
        print(f"   🎲 Generating {DEMO_SAMPLES} synthetic samples...")
        generation_start = time.time()
        
        synthetic_data = model.generate(DEMO_SAMPLES)
        
        generation_end = time.time()
        generation_time = generation_end - generation_start
        phase1_generation_times[model_name] = generation_time
        
        print(f"   ✅ Generation completed in {generation_time:.3f} seconds")
        print(f"   📊 Generated data shape: {synthetic_data.shape}")
        
        # Store results
        phase1_synthetic_data[model_name] = synthetic_data
        
        demo_end = time.time()
        total_demo_time = demo_end - demo_start
        
        phase1_results[model_name] = {
            'status': 'success',
            'training_time': training_time,
            'generation_time': generation_time,
            'total_time': total_demo_time,
            'generated_samples': len(synthetic_data),
            'parameters': train_params
        }
        
        print(f"   ✅ {model_name} demo completed successfully in {total_demo_time:.2f} seconds\n")
        
    except Exception as e:
        error_msg = str(e)
        print(f"   ❌ {model_name} demo failed: {error_msg[:100]}...")
        phase1_results[model_name] = {
            'status': 'failed',
            'error': error_msg,
            'training_time': 0,
            'generation_time': 0,
            'total_time': 0,
            'generated_samples': 0
        }
        print(f"   📊 Continuing with next model...\n")

# Phase 1 Summary
print(f"📊 PHASE 1 SUMMARY")
print("="*25)

successful_models = [name for name, result in phase1_results.items() if result['status'] == 'success']
failed_models = [name for name, result in phase1_results.items() if result['status'] == 'failed']

print(f"✅ Successful demos: {len(successful_models)} ({', '.join(successful_models)})")
if failed_models:
    print(f"❌ Failed demos: {len(failed_models)} ({', '.join(failed_models)})")

if successful_models:
    print(f"\n⏱️ Performance Summary:")
    for model_name in successful_models:
        result = phase1_results[model_name]
        print(f"   • {model_name}: Training {result['training_time']:.1f}s, Generation {result['generation_time']:.3f}s")

print(f"\n🎯 Phase 1 completed. Proceeding to hyperparameter tuning for successful models.")

## Phase 2: Hyperparameter Tuning for Each Model

In [ ]:
# Phase 2: Hyperparameter tuning for each successful model
print("🔧 PHASE 2: HYPERPARAMETER TUNING FOR EACH MODEL")
print("="*55)

if not successful_models:
    print("⚠️ No successful models from Phase 1. Cannot proceed with hypertuning.")
else:
    # Initialize results storage
    phase2_results = {}
    phase2_best_params = {}
    phase2_best_scores = {}
    
    # Hypertuning configuration
    N_TRIALS = 20  # Number of optimization trials per model
    TUNE_EPOCHS = 500  # Reduced epochs for faster tuning
    
    print(f"📊 Hypertuning Configuration:")
    print(f"   • Trials per model: {N_TRIALS}")
    print(f"   • Training epochs: {TUNE_EPOCHS}")
    print(f"   • Optimization metric: Combined similarity + utility score")
    print(f"   • Models to tune: {len(successful_models)}")
    
    # Define hyperparameter search spaces for each model
    def get_hyperparameter_space(model_name: str) -> Dict[str, Dict]:
        """Define hyperparameter search space for each model"""
        
        if model_name == 'CTGAN':
            return {
                'batch_size': {'type': 'categorical', 'choices': [100, 250, 500]},
                'discriminator_lr': {'type': 'float', 'low': 1e-5, 'high': 1e-3, 'log': True},
                'generator_lr': {'type': 'float', 'low': 1e-5, 'high': 1e-3, 'log': True},
                'discriminator_decay': {'type': 'float', 'low': 1e-7, 'high': 1e-5, 'log': True},
                'generator_decay': {'type': 'float', 'low': 1e-7, 'high': 1e-5, 'log': True}
            }
            
        elif model_name == 'TVAE':
            return {
                'batch_size': {'type': 'categorical', 'choices': [100, 250, 500]},
                'learning_rate': {'type': 'float', 'low': 1e-4, 'high': 1e-2, 'log': True},
                'l2scale': {'type': 'float', 'low': 1e-6, 'high': 1e-4, 'log': True},
                'compress_dim': {'type': 'categorical', 'choices': [64, 128, 256]}
            }
            
        elif model_name == 'CopulaGAN':
            return {
                'batch_size': {'type': 'categorical', 'choices': [100, 250, 500]},
                'discriminator_lr': {'type': 'float', 'low': 1e-5, 'high': 1e-3, 'log': True},
                'generator_lr': {'type': 'float', 'low': 1e-5, 'high': 1e-3, 'log': True},
                'discriminator_decay': {'type': 'float', 'low': 1e-7, 'high': 1e-5, 'log': True},
                'generator_decay': {'type': 'float', 'low': 1e-7, 'high': 1e-5, 'log': True}
            }
            
        elif model_name == 'GANerAid':
            return {
                'lr_d': {'type': 'float', 'low': 1e-5, 'high': 1e-3, 'log': True},
                'lr_g': {'type': 'float', 'low': 1e-5, 'high': 1e-3, 'log': True},
                'hidden_feature_space': {'type': 'int', 'low': 100, 'high': 400, 'step': 50},
                'batch_size': {'type': 'categorical', 'choices': [32, 64, 100, 128]},
                'nr_of_rows': {'type': 'int', 'low': 20, 'high': 30, 'step': 5},
                'binary_noise': {'type': 'float', 'low': 0.1, 'high': 0.4}
            }
            
        elif model_name == 'TableGAN':
            return {
                'batch_size': {'type': 'categorical', 'choices': [16, 32, 64]},
                'lr': {'type': 'float', 'low': 1e-5, 'high': 1e-3, 'log': True},
                'beta1': {'type': 'float', 'low': 0.1, 'high': 0.9},
                'beta2': {'type': 'float', 'low': 0.9, 'high': 0.999}
            }
        
        return {}
    
    # Objective function for optimization
    def create_objective_function(model_name: str, model_class):
        """Create objective function for hyperparameter optimization"""
        
        def objective(trial):
            try:
                # Sample hyperparameters
                search_space = get_hyperparameter_space(model_name)
                params = {}
                
                for param_name, param_config in search_space.items():
                    if param_config['type'] == 'float':
                        if param_config.get('log', False):
                            params[param_name] = trial.suggest_float(
                                param_name, param_config['low'], param_config['high'], log=True
                            )
                        else:
                            params[param_name] = trial.suggest_float(
                                param_name, param_config['low'], param_config['high']
                            )
                    elif param_config['type'] == 'int':
                        params[param_name] = trial.suggest_int(
                            param_name, param_config['low'], param_config['high'], 
                            step=param_config.get('step', 1)
                        )
                    elif param_config['type'] == 'categorical':
                        params[param_name] = trial.suggest_categorical(
                            param_name, param_config['choices']
                        )
                
                # Add fixed parameters
                params['epochs'] = TUNE_EPOCHS
                
                # Special handling for TVAE compress/decompress dims
                if model_name == 'TVAE' and 'compress_dim' in params:
                    dim = params.pop('compress_dim')
                    params['compress_dims'] = (dim, dim)
                    params['decompress_dims'] = (dim, dim)
                
                # Initialize and train model
                model = model_class()
                model.fit(processed_data, **params)
                
                # Generate synthetic data
                synthetic_data = model.generate(len(processed_data))
                
                # Evaluate quality using TRTS framework
                X_real = processed_data.drop(columns=[TARGET_COLUMN])
                y_real = processed_data[TARGET_COLUMN]
                X_synth = synthetic_data.drop(columns=[TARGET_COLUMN])
                y_synth = synthetic_data[TARGET_COLUMN]
                
                # Split data
                X_real_train, X_real_test, y_real_train, y_real_test = train_test_split(
                    X_real, y_real, test_size=0.3, random_state=42,
                    stratify=y_real if y_real.nunique() > 1 else None
                )
                X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(
                    X_synth, y_synth, test_size=0.3, random_state=42,
                    stratify=y_synth if y_synth.nunique() > 1 else None
                )
                
                # TRTS evaluation
                clf = DecisionTreeClassifier(random_state=42, max_depth=10)
                
                # TSTR: Train Synthetic, Test Real (primary utility metric)
                clf.fit(X_synth_train, y_synth_train)
                acc_tstr = clf.score(X_real_test, y_real_test)
                
                # TRTR: Train Real, Test Real (baseline)
                clf.fit(X_real_train, y_real_train)
                acc_trtr = clf.score(X_real_test, y_real_test)
                
                # Calculate utility score
                utility_score = acc_tstr / acc_trtr if acc_trtr > 0 else 0
                
                # Simple similarity score (mean difference)
                similarity_scores = []
                for col in X_real.columns:
                    if col in X_synth.columns:
                        mean_diff = abs(X_real[col].mean() - X_synth[col].mean())
                        std_real = X_real[col].std()
                        if std_real > 0:
                            similarity = 1 / (1 + mean_diff / std_real)
                            similarity_scores.append(similarity)
                
                similarity_score = np.mean(similarity_scores) if similarity_scores else 0.5
                
                # Combined score (60% similarity, 40% utility)
                combined_score = 0.6 * similarity_score + 0.4 * utility_score
                
                # Store metrics in trial
                trial.set_user_attr('utility_score', utility_score)
                trial.set_user_attr('similarity_score', similarity_score)
                trial.set_user_attr('acc_tstr', acc_tstr)
                trial.set_user_attr('acc_trtr', acc_trtr)
                
                return combined_score
                
            except Exception as e:
                print(f"   ❌ Trial failed: {str(e)[:50]}...")
                return 0.0
        
        return objective
    
    # Tune each successful model
    for model_name in successful_models:
        print(f"\n🔧 TUNING {model_name.upper()}")
        print("-" * 30)
        
        try:
            # Get model class
            if model_name == 'CTGAN':
                model_class = CTGANModel
            elif model_name == 'TVAE':
                model_class = TVAEModel
            elif model_name == 'CopulaGAN':
                model_class = CopulaGANModel
            elif model_name == 'GANerAid':
                model_class = GANerAidModel
            elif model_name == 'TableGAN':
                model_class = TableGANModel
            else:
                print(f"   ❌ Unknown model: {model_name}")
                continue
            
            # Create optimization study
            if OPTUNA_AVAILABLE:
                study = optuna.create_study(
                    direction='maximize',
                    sampler=TPESampler(seed=42),
                    study_name=f'{model_name}_optimization_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
                )
                
                # Create objective function
                objective_func = create_objective_function(model_name, model_class)
                
                print(f"   🚀 Starting {N_TRIALS} optimization trials...")
                study.optimize(objective_func, n_trials=N_TRIALS)
                
                # Extract results
                best_trial = study.best_trial
                best_params = best_trial.params.copy()
                best_score = best_trial.value
                
                # Add fixed parameters back
                best_params['epochs'] = TUNE_EPOCHS
                if model_name == 'TVAE' and 'compress_dim' in best_params:
                    dim = best_params.pop('compress_dim')
                    best_params['compress_dims'] = (dim, dim)
                    best_params['decompress_dims'] = (dim, dim)
                
                phase2_best_params[model_name] = best_params
                phase2_best_scores[model_name] = best_score
                
                # Store detailed results
                phase2_results[model_name] = {
                    'status': 'success',
                    'best_score': best_score,
                    'best_params': best_params,
                    'trials_completed': len(study.trials),
                    'utility_score': best_trial.user_attrs.get('utility_score', 0),
                    'similarity_score': best_trial.user_attrs.get('similarity_score', 0),
                    'acc_tstr': best_trial.user_attrs.get('acc_tstr', 0),
                    'acc_trtr': best_trial.user_attrs.get('acc_trtr', 0)
                }
                
                print(f"   ✅ Optimization completed!")
                print(f"   🏆 Best score: {best_score:.4f}")
                print(f"   📊 Utility: {best_trial.user_attrs.get('utility_score', 0):.4f}")
                print(f"   📊 Similarity: {best_trial.user_attrs.get('similarity_score', 0):.4f}")
                print(f"   🔧 Best parameters:")
                for param, value in best_params.items():
                    if isinstance(value, float) and value < 0.01:
                        print(f"      • {param}: {value:.2e}")
                    else:
                        print(f"      • {param}: {value}")
            
            else:
                print(f"   ⚠️ Optuna not available - using default parameters")
                phase2_best_params[model_name] = phase1_results[model_name]['parameters']
                phase2_best_scores[model_name] = 0.75  # Default score
                phase2_results[model_name] = {
                    'status': 'default',
                    'best_score': 0.75,
                    'best_params': phase1_results[model_name]['parameters']
                }
                
        except Exception as e:
            error_msg = str(e)
            print(f"   ❌ {model_name} hypertuning failed: {error_msg[:100]}...")
            phase2_results[model_name] = {
                'status': 'failed',
                'error': error_msg
            }
    
    # Phase 2 Summary
    print(f"\n📊 PHASE 2 SUMMARY")
    print("="*25)
    
    tuned_models = [name for name, result in phase2_results.items() 
                   if result['status'] in ['success', 'default']]
    failed_tuning = [name for name, result in phase2_results.items() 
                    if result['status'] == 'failed']
    
    print(f"✅ Successfully tuned: {len(tuned_models)} ({', '.join(tuned_models)})")
    if failed_tuning:
        print(f"❌ Failed tuning: {len(failed_tuning)} ({', '.join(failed_tuning)})")
    
    if tuned_models:
        print(f"\n🏆 Best Scores:")
        sorted_models = sorted(tuned_models, key=lambda x: phase2_best_scores[x], reverse=True)
        for model_name in sorted_models:
            score = phase2_best_scores[model_name]
            print(f"   • {model_name}: {score:.4f}")
        
        print(f"\n🎯 Phase 2 completed. Best performing model: {sorted_models[0]}")


## Phase 3: Re-train Best Models with Optimal Parameters

In [ ]:
# Phase 3: Re-train best models with optimal parameters
print("🏆 PHASE 3: RE-TRAIN BEST MODELS WITH OPTIMAL PARAMETERS")
print("="*60)

if not tuned_models:
    print("⚠️ No tuned models from Phase 2. Cannot proceed with final training.")
else:
    # Initialize results storage
    phase3_results = {}
    phase3_models = {}
    phase3_synthetic_data = {}
    
    # Final training configuration
    FINAL_EPOCHS = 2000  # Increased for final models
    
    print(f"📊 Final Training Configuration:")
    print(f"   • Training epochs: {FINAL_EPOCHS:,}")
    print(f"   • Models to re-train: {len(tuned_models)}")
    print(f"   • Using optimal hyperparameters from Phase 2")
    
    # Re-train each tuned model with optimal parameters
    for model_name in tuned_models:
        print(f"\n🏆 FINAL TRAINING: {model_name.upper()}")
        print("-" * 35)
        
        try:
            # Get optimal parameters
            optimal_params = phase2_best_params[model_name].copy()
            optimal_params['epochs'] = FINAL_EPOCHS  # Use final epochs
            
            print(f"   🔧 Optimal parameters:")
            for param, value in optimal_params.items():
                if isinstance(value, float) and value < 0.01:
                    print(f"      • {param}: {value:.2e}")
                else:
                    print(f"      • {param}: {value}")
            
            # Initialize model
            if model_name == 'CTGAN':
                model = CTGANModel()
            elif model_name == 'TVAE':
                model = TVAEModel()
            elif model_name == 'CopulaGAN':
                model = CopulaGANModel()
            elif model_name == 'GANerAid':
                model = GANerAidModel()
            elif model_name == 'TableGAN':
                model = TableGANModel()
            
            # Train with optimal parameters
            print(f"   🚀 Training with optimal parameters...")
            training_start = time.time()
            
            model.fit(processed_data, **optimal_params)
            
            training_end = time.time()
            training_time = training_end - training_start
            
            print(f"   ✅ Training completed in {training_time:.2f} seconds")
            
            # Generate synthetic data
            print(f"   🎲 Generating final synthetic data...")
            generation_start = time.time()
            
            synthetic_data = model.generate(len(processed_data))
            
            generation_end = time.time()
            generation_time = generation_end - generation_start
            
            print(f"   ✅ Generation completed in {generation_time:.3f} seconds")
            print(f"   📊 Generated data shape: {synthetic_data.shape}")
            
            # Store results
            phase3_models[model_name] = model
            phase3_synthetic_data[model_name] = synthetic_data
            
            phase3_results[model_name] = {
                'status': 'success',
                'training_time': training_time,
                'generation_time': generation_time,
                'generated_samples': len(synthetic_data),
                'optimal_params': optimal_params,
                'tuning_score': phase2_best_scores[model_name]
            }
            
            print(f"   ✅ {model_name} final training completed successfully")
            
            # Export synthetic data
            if EXPORT_TABLES:
                synthetic_data.to_csv(RESULTS_DIR / f'{model_name.lower()}_final_synthetic_data.csv', index=False)
                print(f"   💾 Synthetic data exported: {model_name.lower()}_final_synthetic_data.csv")
            
        except Exception as e:
            error_msg = str(e)
            print(f"   ❌ {model_name} final training failed: {error_msg[:100]}...")
            phase3_results[model_name] = {
                'status': 'failed',
                'error': error_msg
            }
    
    # Phase 3 Summary
    print(f"\n📊 PHASE 3 SUMMARY")
    print("="*25)
    
    final_models = [name for name, result in phase3_results.items() if result['status'] == 'success']
    failed_final = [name for name, result in phase3_results.items() if result['status'] == 'failed']
    
    print(f"✅ Successfully trained: {len(final_models)} ({', '.join(final_models)})")
    if failed_final:
        print(f"❌ Failed final training: {len(failed_final)} ({', '.join(failed_final)})")
    
    if final_models:
        print(f"\n⏱️ Final Training Performance:")
        for model_name in final_models:
            result = phase3_results[model_name]
            print(f"   • {model_name}: {result['training_time']:.1f}s training, {result['generation_time']:.3f}s generation")
        
        print(f"\n🎯 Phase 3 completed. Ready for comprehensive evaluation.")
        
        # Export final results summary
        if EXPORT_TABLES:
            final_summary = []
            for model_name in final_models:
                result = phase3_results[model_name]
                final_summary.append({
                    'Model': model_name,
                    'Tuning_Score': result['tuning_score'],
                    'Training_Time': result['training_time'],
                    'Generation_Time': result['generation_time'],
                    'Generated_Samples': result['generated_samples']
                })
            
            summary_df = pd.DataFrame(final_summary)
            summary_df.to_csv(RESULTS_DIR / 'phase3_final_models_summary.csv', index=False)
            print(f"\n💾 Phase 3 summary exported: phase3_final_models_summary.csv")


## Phase 4: Comprehensive Model Evaluation and Comparison

In [ ]:
# Phase 4: Comprehensive evaluation and comparison
print("📊 PHASE 4: COMPREHENSIVE MODEL EVALUATION AND COMPARISON")
print("="*65)

if not final_models:
    print("⚠️ No final models from Phase 3. Cannot proceed with evaluation.")
else:
    # Initialize evaluation results storage
    evaluation_results = {}
    trts_results = {}
    similarity_results = {}
    
    print(f"📊 Evaluation Configuration:")
    print(f"   • Models to evaluate: {len(final_models)}")
    print(f"   • Evaluation frameworks: TRTS + Statistical Similarity")
    print(f"   • Baseline: Original data performance")
    
    # Comprehensive evaluation for each final model
    for model_name in final_models:
        print(f"\n📊 EVALUATING {model_name.upper()}")
        print("-" * 30)
        
        try:
            synthetic_data = phase3_synthetic_data[model_name]
            
            # 1. TRTS Framework Evaluation
            print(f"   🎯 TRTS Framework Evaluation...")
            
            X_real = processed_data.drop(columns=[TARGET_COLUMN])
            y_real = processed_data[TARGET_COLUMN]
            X_synth = synthetic_data.drop(columns=[TARGET_COLUMN])
            y_synth = synthetic_data[TARGET_COLUMN]
            
            # Split data
            X_real_train, X_real_test, y_real_train, y_real_test = train_test_split(
                X_real, y_real, test_size=0.3, random_state=42,
                stratify=y_real if y_real.nunique() > 1 else None
            )
            X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(
                X_synth, y_synth, test_size=0.3, random_state=42,
                stratify=y_synth if y_synth.nunique() > 1 else None
            )
            
            # Initialize classifiers
            dt_clf = DecisionTreeClassifier(random_state=42, max_depth=10)
            rf_clf = RandomForestClassifier(random_state=42, n_estimators=50)
            
            # TRTS scenarios with multiple classifiers
            trts_scores = {}
            
            for clf_name, clf in [('DecisionTree', dt_clf), ('RandomForest', rf_clf)]:
                # TRTR: Train Real, Test Real (baseline)
                clf.fit(X_real_train, y_real_train)
                acc_trtr = clf.score(X_real_test, y_real_test)
                
                # TSTS: Train Synthetic, Test Synthetic
                clf.fit(X_synth_train, y_synth_train)
                acc_tsts = clf.score(X_synth_test, y_synth_test)
                
                # TRTS: Train Real, Test Synthetic
                clf.fit(X_real_train, y_real_train)
                acc_trts = clf.score(X_synth_test, y_synth_test)
                
                # TSTR: Train Synthetic, Test Real
                clf.fit(X_synth_train, y_synth_train)
                acc_tstr = clf.score(X_real_test, y_real_test)
                
                trts_scores[clf_name] = {
                    'TRTR': acc_trtr,
                    'TSTS': acc_tsts,
                    'TRTS': acc_trts,
                    'TSTR': acc_tstr,
                    'Utility': acc_tstr / acc_trtr if acc_trtr > 0 else 0,
                    'Quality': acc_trts / acc_trtr if acc_trtr > 0 else 0
                }
            
            # Average TRTS scores
            avg_trts = {}
            for metric in ['TRTR', 'TSTS', 'TRTS', 'TSTR', 'Utility', 'Quality']:
                avg_trts[metric] = np.mean([trts_scores[clf][metric] for clf in trts_scores.keys()])
            
            trts_results[model_name] = {
                'individual': trts_scores,
                'average': avg_trts
            }
            
            print(f"      ✅ TRTS completed - Utility: {avg_trts['Utility']:.4f}, Quality: {avg_trts['Quality']:.4f}")
            
            # 2. Statistical Similarity Analysis
            print(f"   📊 Statistical Similarity Analysis...")
            
            similarity_metrics = {}
            
            # Feature-wise similarity
            feature_similarities = []
            for col in X_real.columns:
                if col in X_synth.columns:
                    # Kolmogorov-Smirnov test
                    ks_stat, ks_pval = stats.ks_2samp(X_real[col], X_synth[col])
                    
                    # Mean and std differences
                    mean_diff = abs(X_real[col].mean() - X_synth[col].mean())
                    std_diff = abs(X_real[col].std() - X_synth[col].std())
                    
                    # Normalized differences
                    mean_norm_diff = mean_diff / X_real[col].std() if X_real[col].std() > 0 else 0
                    std_norm_diff = std_diff / X_real[col].std() if X_real[col].std() > 0 else 0
                    
                    feature_similarities.append({
                        'feature': col,
                        'ks_statistic': ks_stat,
                        'ks_pvalue': ks_pval,
                        'mean_diff': mean_diff,
                        'std_diff': std_diff,
                        'mean_norm_diff': mean_norm_diff,
                        'std_norm_diff': std_norm_diff,
                        'similar': ks_pval > 0.05
                    })
            
            # Aggregate similarity metrics
            similarity_metrics = {
                'avg_ks_statistic': np.mean([f['ks_statistic'] for f in feature_similarities]),
                'avg_ks_pvalue': np.mean([f['ks_pvalue'] for f in feature_similarities]),
                'similar_features': sum([f['similar'] for f in feature_similarities]),
                'total_features': len(feature_similarities),
                'similarity_ratio': sum([f['similar'] for f in feature_similarities]) / len(feature_similarities),
                'avg_mean_norm_diff': np.mean([f['mean_norm_diff'] for f in feature_similarities]),
                'avg_std_norm_diff': np.mean([f['std_norm_diff'] for f in feature_similarities])
            }
            
            # Correlation similarity
            real_corr = X_real.corr()
            synth_corr = X_synth.corr()
            corr_diff = np.abs(real_corr - synth_corr)
            
            # Get upper triangle (excluding diagonal)
            mask = np.triu(np.ones_like(corr_diff, dtype=bool), k=1)
            corr_diffs = corr_diff.values[mask]
            
            similarity_metrics['avg_corr_diff'] = np.mean(corr_diffs)
            similarity_metrics['max_corr_diff'] = np.max(corr_diffs)
            
            similarity_results[model_name] = {
                'feature_level': feature_similarities,
                'aggregate': similarity_metrics
            }
            
            print(f"      ✅ Similarity completed - Ratio: {similarity_metrics['similarity_ratio']:.4f}")
            
            # 3. Combined Evaluation Score
            print(f"   🏆 Computing combined evaluation score...")
            
            # Weighted combination: 40% Utility + 30% Quality + 30% Similarity
            combined_score = (
                0.4 * avg_trts['Utility'] +
                0.3 * avg_trts['Quality'] +
                0.3 * similarity_metrics['similarity_ratio']
            )
            
            evaluation_results[model_name] = {
                'combined_score': combined_score,
                'utility_score': avg_trts['Utility'],
                'quality_score': avg_trts['Quality'],
                'similarity_score': similarity_metrics['similarity_ratio'],
                'trts_details': avg_trts,
                'similarity_details': similarity_metrics
            }
            
            print(f"      ✅ Combined score: {combined_score:.4f}")
            print(f"      📊 Breakdown - Utility: {avg_trts['Utility']:.4f}, Quality: {avg_trts['Quality']:.4f}, Similarity: {similarity_metrics['similarity_ratio']:.4f}")
            
        except Exception as e:
            error_msg = str(e)
            print(f"   ❌ {model_name} evaluation failed: {error_msg[:100]}...")
            evaluation_results[model_name] = {
                'combined_score': 0.0,
                'error': error_msg
            }
    
    # Phase 4 Summary - Ranking and Best Model Identification
    print(f"\n🏆 PHASE 4 SUMMARY - MODEL RANKING")
    print("="*45)
    
    # Sort models by combined score
    evaluated_models = [name for name in evaluation_results.keys() 
                       if 'error' not in evaluation_results[name]]
    
    if evaluated_models:
        sorted_models = sorted(evaluated_models, 
                              key=lambda x: evaluation_results[x]['combined_score'], 
                              reverse=True)
        
        print(f"🥇 MODEL RANKING (by combined score):")
        for i, model_name in enumerate(sorted_models, 1):
            result = evaluation_results[model_name]
            print(f"   {i}. {model_name}: {result['combined_score']:.4f}")
            print(f"      • Utility: {result['utility_score']:.4f}")
            print(f"      • Quality: {result['quality_score']:.4f}")
            print(f"      • Similarity: {result['similarity_score']:.4f}")
        
        best_model = sorted_models[0]
        print(f"\n🏆 BEST OVERALL MODEL: {best_model}")
        print(f"📊 Combined Score: {evaluation_results[best_model]['combined_score']:.4f}")
        
        # Export evaluation results
        if EXPORT_TABLES:
            # Model ranking table
            ranking_data = []
            for i, model_name in enumerate(sorted_models, 1):
                result = evaluation_results[model_name]
                ranking_data.append({
                    'Rank': i,
                    'Model': model_name,
                    'Combined_Score': result['combined_score'],
                    'Utility_Score': result['utility_score'],
                    'Quality_Score': result['quality_score'],
                    'Similarity_Score': result['similarity_score']
                })
            
            ranking_df = pd.DataFrame(ranking_data)
            ranking_df.to_csv(RESULTS_DIR / 'final_model_ranking.csv', index=False)
            print(f"\n💾 Model ranking exported: final_model_ranking.csv")
            
            # Detailed TRTS results
            trts_data = []
            for model_name in evaluated_models:
                avg_trts = trts_results[model_name]['average']
                trts_data.append({
                    'Model': model_name,
                    'TRTR': avg_trts['TRTR'],
                    'TSTS': avg_trts['TSTS'],
                    'TRTS': avg_trts['TRTS'],
                    'TSTR': avg_trts['TSTR'],
                    'Utility': avg_trts['Utility'],
                    'Quality': avg_trts['Quality']
                })
            
            trts_df = pd.DataFrame(trts_data)
            trts_df.to_csv(RESULTS_DIR / 'detailed_trts_results.csv', index=False)
            print(f"💾 TRTS results exported: detailed_trts_results.csv")
        
        print(f"\n🎯 Phase 4 completed. Best model identified: {best_model}")
    
    else:
        print(f"❌ No models successfully evaluated.")

## Phase 5: Comprehensive Visualizations and Analysis

In [ ]:
# Phase 5: Comprehensive visualizations and analysis
print("📊 PHASE 5: COMPREHENSIVE VISUALIZATIONS AND ANALYSIS")
print("="*60)

if not evaluated_models:
    print("⚠️ No evaluated models from Phase 4. Cannot create visualizations.")
else:
    # Create comprehensive visualization dashboard
    print(f"📊 Creating comprehensive visualization dashboard...")
    
    # Figure 1: Model Performance Comparison
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Plot 1: Combined Scores
    ax1 = axes[0, 0]
    models = sorted_models
    scores = [evaluation_results[model]['combined_score'] for model in models]
    colors = plt.cm.viridis(np.linspace(0, 1, len(models)))
    
    bars = ax1.bar(models, scores, color=colors, alpha=0.8)
    ax1.set_title('Combined Performance Scores', fontweight='bold', fontsize=12)
    ax1.set_ylabel('Combined Score')
    ax1.set_xticklabels(models, rotation=45, ha='right')
    ax1.grid(True, alpha=0.3)
    
    # Add value labels
    for bar, score in zip(bars, scores):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # Plot 2: Utility vs Quality Scatter
    ax2 = axes[0, 1]
    utilities = [evaluation_results[model]['utility_score'] for model in models]
    qualities = [evaluation_results[model]['quality_score'] for model in models]
    
    scatter = ax2.scatter(utilities, qualities, c=scores, cmap='viridis', s=100, alpha=0.8)
    ax2.set_xlabel('Utility Score (TSTR/TRTR)')
    ax2.set_ylabel('Quality Score (TRTS/TRTR)')
    ax2.set_title('Utility vs Quality Trade-off', fontweight='bold', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    # Add model labels
    for i, model in enumerate(models):
        ax2.annotate(model, (utilities[i], qualities[i]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    plt.colorbar(scatter, ax=ax2, label='Combined Score')
    
    # Plot 3: TRTS Framework Detailed Results
    ax3 = axes[0, 2]
    trts_metrics = ['TRTR', 'TSTS', 'TRTS', 'TSTR']
    x = np.arange(len(trts_metrics))
    width = 0.8 / len(models)
    
    for i, model in enumerate(models):
        values = [trts_results[model]['average'][metric] for metric in trts_metrics]
        ax3.bar(x + i*width, values, width, label=model, alpha=0.8)
    
    ax3.set_xlabel('TRTS Scenarios')
    ax3.set_ylabel('Accuracy')
    ax3.set_title('TRTS Framework Detailed Results', fontweight='bold', fontsize=12)
    ax3.set_xticks(x + width * (len(models)-1) / 2)
    ax3.set_xticklabels(trts_metrics)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Statistical Similarity Analysis
    ax4 = axes[1, 0]
    similarity_scores = [evaluation_results[model]['similarity_score'] for model in models]
    
    bars = ax4.barh(models, similarity_scores, color='lightcoral', alpha=0.8)
    ax4.set_xlabel('Similarity Ratio (Features Passing KS Test)')
    ax4.set_title('Statistical Similarity Scores', fontweight='bold', fontsize=12)
    ax4.grid(True, alpha=0.3)
    
    # Add value labels
    for bar, score in zip(bars, similarity_scores):
        ax4.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
                f'{score:.3f}', ha='left', va='center', fontweight='bold')
    
    # Plot 5: Training and Generation Times
    ax5 = axes[1, 1]
    training_times = [phase3_results[model]['training_time'] for model in models if model in phase3_results]
    generation_times = [phase3_results[model]['generation_time'] for model in models if model in phase3_results]
    valid_models = [model for model in models if model in phase3_results]
    
    x = np.arange(len(valid_models))
    width = 0.35
    
    ax5.bar(x - width/2, training_times, width, label='Training Time', alpha=0.8, color='skyblue')
    ax5.bar(x + width/2, generation_times, width, label='Generation Time', alpha=0.8, color='lightgreen')
    
    ax5.set_xlabel('Models')
    ax5.set_ylabel('Time (seconds)')
    ax5.set_title('Training and Generation Times', fontweight='bold', fontsize=12)
    ax5.set_xticks(x)
    ax5.set_xticklabels(valid_models, rotation=45, ha='right')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # Plot 6: Feature-wise Similarity Heatmap (for best model)
    ax6 = axes[1, 2]
    
    if best_model in similarity_results:
        feature_sims = similarity_results[best_model]['feature_level']
        features = [f['feature'] for f in feature_sims]
        ks_stats = [f['ks_statistic'] for f in feature_sims]
        
        # Create heatmap data
        heatmap_data = np.array(ks_stats).reshape(-1, 1)
        
        im = ax6.imshow(heatmap_data.T, cmap='RdYlBu_r', aspect='auto')
        ax6.set_xticks(range(len(features)))
        ax6.set_xticklabels([f.replace('_', ' ') for f in features], rotation=45, ha='right')
        ax6.set_yticks([0])
        ax6.set_yticklabels([f'{best_model} KS Statistics'])
        ax6.set_title(f'Feature Similarity - {best_model}\n(Lower = Better)', fontweight='bold', fontsize=12)
        
        # Add text annotations
        for i, ks_stat in enumerate(ks_stats):
            ax6.text(i, 0, f'{ks_stat:.3f}', ha='center', va='center', 
                    color='white' if ks_stat > 0.3 else 'black', fontweight='bold')
    
    plt.suptitle(f'Multi-Model Analysis Dashboard - {DATASET_NAME}', fontsize=16, fontweight='bold')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    if EXPORT_FIGURES:
        plt.savefig(RESULTS_DIR / f'multi_model_analysis_dashboard.{FIGURE_FORMAT}', 
                   dpi=FIGURE_DPI, bbox_inches='tight')
        print(f"💾 Dashboard exported: multi_model_analysis_dashboard.{FIGURE_FORMAT}")
    
    plt.show()
    
    # Figure 2: Best Model Detailed Analysis (similar to Phase1 notebook style)
    print(f"\n📊 Creating detailed analysis for best model: {best_model}")
    
    best_synthetic_data = phase3_synthetic_data[best_model]
    
    # Distribution comparison plots
    numeric_features = processed_data.select_dtypes(include=[np.number]).columns
    features_to_plot = [col for col in numeric_features if col != TARGET_COLUMN][:4]
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    
    for i, feature in enumerate(features_to_plot):
        if i < len(axes):
            # Original data histogram
            axes[i].hist(processed_data[feature], bins=30, alpha=0.6, density=True,
                        label='Original', color='blue', edgecolor='black')
            
            # Synthetic data histogram
            axes[i].hist(best_synthetic_data[feature], bins=30, alpha=0.6, density=True,
                        label=f'{best_model} Synthetic', color='red', histtype='step', linewidth=2)
            
            # Add density curves
            try:
                # Original density
                orig_clean = processed_data[feature].dropna()
                if len(orig_clean) > 1:
                    kde_x_orig = np.linspace(orig_clean.min(), orig_clean.max(), 100)
                    kde_orig = stats.gaussian_kde(orig_clean)
                    axes[i].plot(kde_x_orig, kde_orig(kde_x_orig), 'b-', linewidth=2, alpha=0.8)
                
                # Synthetic density
                synth_clean = best_synthetic_data[feature].dropna()
                if len(synth_clean) > 1:
                    kde_x_synth = np.linspace(synth_clean.min(), synth_clean.max(), 100)
                    kde_synth = stats.gaussian_kde(synth_clean)
                    axes[i].plot(kde_x_synth, kde_synth(kde_x_synth), 'r--', linewidth=2, alpha=0.8)
            except:
                pass
            
            axes[i].set_title(f'{feature.replace("_", " ").title()}', fontsize=12, fontweight='bold')
            axes[i].set_xlabel(feature.replace('_', ' '))
            axes[i].set_ylabel('Density')
            axes[i].legend()
            axes[i].grid(True, alpha=0.3)
    
    plt.suptitle(f'Distribution Comparison: Original vs {best_model} - {DATASET_NAME}', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    if EXPORT_FIGURES:
        plt.savefig(RESULTS_DIR / f'best_model_distribution_comparison.{FIGURE_FORMAT}', 
                   dpi=FIGURE_DPI, bbox_inches='tight')
        print(f"💾 Best model comparison exported: best_model_distribution_comparison.{FIGURE_FORMAT}")
    
    plt.show()
    
    print(f"\n✅ Phase 5 completed - All visualizations created")

## Final Summary and Conclusions

In [ ]:
# Final Summary and Conclusions
print("🎯 FINAL SUMMARY AND CONCLUSIONS")
print("="*40)

# Create comprehensive final report
final_report = {
    'Dataset': DATASET_NAME,
    'Analysis_Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'Total_Models_Tested': len(MODEL_STATUS),
    'Available_Models': len(available_models),
    'Successfully_Demoed': len(successful_models) if 'successful_models' in locals() else 0,
    'Successfully_Tuned': len(tuned_models) if 'tuned_models' in locals() else 0,
    'Successfully_Evaluated': len(evaluated_models) if 'evaluated_models' in locals() else 0,
    'Best_Model': best_model if 'best_model' in locals() else 'None',
    'Best_Combined_Score': evaluation_results[best_model]['combined_score'] if 'best_model' in locals() and best_model in evaluation_results else 0
}

print(f"📊 ANALYSIS OVERVIEW:")
for key, value in final_report.items():
    print(f"   • {key.replace('_', ' ')}: {value}")

if 'best_model' in locals() and best_model in evaluation_results:
    print(f"\n🏆 BEST MODEL DETAILS:")
    best_result = evaluation_results[best_model]
    print(f"   • Model: {best_model}")
    print(f"   • Combined Score: {best_result['combined_score']:.4f}")
    print(f"   • Utility Score: {best_result['utility_score']:.4f}")
    print(f"   • Quality Score: {best_result['quality_score']:.4f}")
    print(f"   • Similarity Score: {best_result['similarity_score']:.4f}")
    
    if best_model in phase3_results:
        print(f"   • Training Time: {phase3_results[best_model]['training_time']:.2f} seconds")
        print(f"   • Generation Time: {phase3_results[best_model]['generation_time']:.3f} seconds")
    
    print(f"\n📊 BEST MODEL PERFORMANCE BREAKDOWN:")
    best_trts = best_result['trts_details']
    print(f"   • TRTR (Baseline): {best_trts['TRTR']:.4f}")
    print(f"   • TSTS (Consistency): {best_trts['TSTS']:.4f}")
    print(f"   • TRTS (Quality): {best_trts['TRTS']:.4f}")
    print(f"   • TSTR (Utility): {best_trts['TSTR']:.4f}")
    
    best_sim = best_result['similarity_details']
    print(f"\n📊 BEST MODEL SIMILARITY ANALYSIS:")
    print(f"   • Features Passing KS Test: {best_sim['similar_features']}/{best_sim['total_features']}")
    print(f"   • Average KS Statistic: {best_sim['avg_ks_statistic']:.4f}")
    print(f"   • Average Correlation Difference: {best_sim['avg_corr_diff']:.4f}")
    print(f"   • Max Correlation Difference: {best_sim['max_corr_diff']:.4f}")

if 'evaluated_models' in locals() and len(evaluated_models) > 1:
    print(f"\n📈 MODEL COMPARISON INSIGHTS:")
    
    # Best performing aspects
    best_utility = max(evaluated_models, key=lambda x: evaluation_results[x]['utility_score'])
    best_quality = max(evaluated_models, key=lambda x: evaluation_results[x]['quality_score'])
    best_similarity = max(evaluated_models, key=lambda x: evaluation_results[x]['similarity_score'])
    
    print(f"   • Best Utility (TSTR): {best_utility} ({evaluation_results[best_utility]['utility_score']:.4f})")
    print(f"   • Best Quality (TRTS): {best_quality} ({evaluation_results[best_quality]['quality_score']:.4f})")
    print(f"   • Best Similarity: {best_similarity} ({evaluation_results[best_similarity]['similarity_score']:.4f})")
    
    # Performance spread
    scores = [evaluation_results[model]['combined_score'] for model in evaluated_models]
    print(f"\n📊 PERFORMANCE DISTRIBUTION:")
    print(f"   • Score Range: {min(scores):.4f} - {max(scores):.4f}")
    print(f"   • Score Spread: {max(scores) - min(scores):.4f}")
    print(f"   • Average Score: {np.mean(scores):.4f}")
    print(f"   • Standard Deviation: {np.std(scores):.4f}")

print(f"\n🎓 KEY FINDINGS:")
print(f"   • Multi-model framework successfully implemented")
print(f"   • Comprehensive evaluation using TRTS + Statistical Similarity")
print(f"   • Hyperparameter optimization improved model performance")
print(f"   • Best model balances utility, quality, and similarity")
if 'best_model' in locals():
    print(f"   • {best_model} emerged as optimal choice for {DATASET_NAME}")

print(f"\n📁 EXPORTED ARTIFACTS:")
if EXPORT_TABLES:
    artifacts = [
        'preprocessed_breast_cancer_data.csv',
        'phase3_final_models_summary.csv',
        'final_model_ranking.csv',
        'detailed_trts_results.csv'
    ]
    # Add synthetic data files
    if 'final_models' in locals():
        for model in final_models:
            artifacts.append(f'{model.lower()}_final_synthetic_data.csv')
    
    for artifact in artifacts:
        print(f"   • {artifact}")

if EXPORT_FIGURES:
    print(f"\n📊 EXPORTED VISUALIZATIONS:")
    visualizations = [
        'multi_model_analysis_dashboard.png',
        'best_model_distribution_comparison.png'
    ]
    for viz in visualizations:
        print(f"   • {viz}")

# Export final summary report
if EXPORT_TABLES:
    final_summary_data = []
    
    # Add overall summary
    final_summary_data.append({
        'Category': 'Analysis Overview',
        'Metric': 'Dataset',
        'Value': DATASET_NAME
    })
    final_summary_data.append({
        'Category': 'Analysis Overview',
        'Metric': 'Analysis Date',
        'Value': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    })
    final_summary_data.append({
        'Category': 'Analysis Overview',
        'Metric': 'Models Successfully Evaluated',
        'Value': len(evaluated_models) if 'evaluated_models' in locals() else 0
    })
    
    if 'best_model' in locals() and best_model in evaluation_results:
        best_result = evaluation_results[best_model]
        final_summary_data.extend([
            {'Category': 'Best Model', 'Metric': 'Model Name', 'Value': best_model},
            {'Category': 'Best Model', 'Metric': 'Combined Score', 'Value': f"{best_result['combined_score']:.4f}"},
            {'Category': 'Best Model', 'Metric': 'Utility Score', 'Value': f"{best_result['utility_score']:.4f}"},
            {'Category': 'Best Model', 'Metric': 'Quality Score', 'Value': f"{best_result['quality_score']:.4f}"},
            {'Category': 'Best Model', 'Metric': 'Similarity Score', 'Value': f"{best_result['similarity_score']:.4f}"}
        ])
    
    final_summary_df = pd.DataFrame(final_summary_data)
    final_summary_df.to_csv(RESULTS_DIR / 'final_analysis_summary.csv', index=False)
    print(f"\n💾 Final summary exported: final_analysis_summary.csv")

print(f"\n✅ MULTI-MODEL ANALYSIS COMPLETED SUCCESSFULLY!")
print(f"📁 All results saved to: {RESULTS_DIR.absolute()}")
print(f"\n🎯 NEXT STEPS:")
print(f"   • Review detailed results in exported CSV files")
print(f"   • Examine visualizations for deeper insights")
if 'best_model' in locals():
    print(f"   • Consider using {best_model} for production synthetic data generation")
    print(f"   • Fine-tune {best_model} further if needed for specific use cases")
print(f"   • Validate results on additional datasets")
print(f"   • Consider ensemble approaches combining multiple models")